### TensorFlow tutorial and some useful Functions  
  
This notebook is a tutorial for TensorFlow. I will explain the basic usage of TensorFlow and some useful functions.  
### Table of contents

1.- Import the libraries  
2.- Load the Data  
3.- Prepare the Data  
4.- Create a Sequential Model  
5.- Create a Model Using Functional API  
6.- Train the Model  
7.- Predict on Your Own Data  
